# Demo: Serenity Derivatives API - Option Pricing

Serenity builds in sophisticated option and rates analytics as part of its core offering, and these functions
are all exposed via the API. This notebook shows how you can use it to price European options on BTC, ETH and SOL.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2
#%run -i init_demo.py

In [ ]:
# to change back to run init_demo.py when ready
config_id = 'athansor'
from serenity_sdk.client import SerenityApiProvider, SerenityClient
from serenity_sdk.config import load_local_config
config = load_local_config(config_id)
client = SerenityClient(config)
api = SerenityApiProvider(client)

In [ ]:
from datetime import datetime, timedelta
from uuid import UUID, uuid4
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from serenity_types.pricing.derivatives.rates.yield_curve import YieldCurveVersion
from serenity_types.pricing.derivatives.options.valuation import DiscountingMethod, OptionValuationRequest, OptionValuation

# helper modules
from serenity_sdk.renderers.derivatives.widget_tools import OptionChooser, YieldCurveVersionTimeChooser, VolatilitySurfaceVersionTimeChooser
from serenity_sdk.renderers.derivatives.table_plot import YieldCurveTablePlot, VolatilitySurfaceTablePlot, OptionValuationResultTablePlot

# plot parameters
plt.rcParams['font.size'] = '16'

In [ ]:
# Pick a datetime. 
now_time = datetime.utcnow() - timedelta(hours=3)

# Read samples of pre-defined options
For now, we read from a csv file. 
In future, we plan to support an API

In [ ]:
# Let's first remind that our asset ids are represented using uuids
# The mapping between asset id and native symbols are as below:
asset_summaries = api.refdata().get_asset_summaries()
asset_summaries = [{key: value for key, value in summary.items() if key != 'xrefSymbols'} for summary in asset_summaries]
asset_summaries = pd.json_normalize(asset_summaries)[['assetId', 'nativeSymbol']]
asset_summaries = asset_summaries[asset_summaries['nativeSymbol'].isin(['BTC','ETH', 'SOL'])]
asset_summaries

In [ ]:
# Load sample options in the system & merge with the underlying asset id and symbol
sample_option_data_file = os.path.join('sample_data', 'list_options_20221202.csv')
sample_options = pd.read_csv(sample_option_data_file, parse_dates=['expiry_datetime'])

sample_option_data_file = os.path.join('sample_data', 'list_options_20221202.csv')
sample_options = pd.read_csv(sample_option_data_file, parse_dates=['expiry_datetime'])\
    [['linked_asset_id', 'native_symbol', 'asset_id', 'option_type', 'expiry_datetime', 'strike_price','option_style']]\
    .sort_values(['linked_asset_id', 'expiry_datetime', 'strike_price'])
sample_options['expiry_datetime'] = sample_options['expiry_datetime'].dt.tz_localize(None)
sample_options = pd.merge(sample_options, asset_summaries, how='left', left_on='linked_asset_id', right_on='assetId')
sample_options.drop('assetId', axis=1, inplace=True)
sample_options.rename(columns={'nativeSymbol':'linked_asset_native_symbol'}, inplace=True)

## Peak samples of predefined options

In [ ]:
sample_options.head(3)

## Select the option to use as a base line

In [ ]:
option_chooser = OptionChooser(sample_options)
print('Select an option to play with')
display(option_chooser.get_widget_to_display())

In [ ]:
# Show the details of the option selected
predefined_option_info = option_chooser.get_selected_option()
predefined_option_info

# Option Valuation

## using a pre-defined option and its replace
* Use the asset_id (uuid) of the pre-defined option to construct a option valuation object
* We constuct the identical option using the attributes of the pre-defined option

In [ ]:
demo0_optvals = {}
demo0_optvals['predefined'] = OptionValuation(
    option_valuation_id=str(uuid4()),
    qty = 10, 
    option_asset_id=UUID(predefined_option_info['asset_id']),
    contract_size=1
)

demo1_optvals = {}
demo1_optvals['predefined'] = demo0_optvals['predefined']
demo1_optvals['predefined_replica'] = OptionValuation(
    option_valuation_id=str(uuid4()),
    qty = 10, 
    underlier_asset_id=predefined_option_info['linked_asset_id'],
    strike=predefined_option_info['strike_price'],
    expiry=predefined_option_info['expiry_datetime'],
    option_type=predefined_option_info['option_type'],
    option_style=predefined_option_info['option_style'],
    contract_size=1)


In [ ]:
def run_compute_option_valuations(the_optvals, as_of_time=None):
    if as_of_time is None: 
        request = OptionValuationRequest(options=[v for v in the_optvals.values()])
    else:
        request = OptionValuationRequest(as_of_time=as_of_time, options=[v for v in the_optvals.values()])
    val_results = api.pricer().compute_option_valuations(request)

    # use a helper object to format output
    ovr_tp = OptionValuationResultTablePlot(val_results, the_optvals)
    return ovr_tp.results_table

In [ ]:
res_table = run_compute_option_valuations(demo1_optvals, now_time)
res_table

## Market Data Overrides

In [ ]:
# market data bump

spot_bumps = np.array([-20.0, -10.0, -5.0, -2.5, -1.0, 0.0, +1.0, +2.5, +5.0, +10.0, +20.0])/100.0 + 1.0

base_optval = demo1_optvals['predefined_replica'].copy()
spot_bumps_optvals = {}
spot_bumps_optvals['base'] = base_optval
for sb in spot_bumps:
    optval_this = base_optval.copy()
    optval_this.option_valuation_id=str(uuid4())
    optval_this.spot_price_bump=sb
    spot_bumps_optvals[f'spot_bump_{sb}'] = optval_this

In [ ]:
res_table = run_compute_option_valuations(spot_bumps_optvals, now_time)
res_table.to_clipboard()

In [ ]:
base_res = res_table['base']
qty = base_res['spot_notional']/base_res['spot_price']
print(f'quantity: {qty}')

In [ ]:
delta_ccy = res_table['base']['delta_ccy']
plt.figure()
plt.plot(spot_bumps, res_table[[c for c in res_table.columns if c!='base']].loc['pv']-res_table['base']['pv'])
plt.plot(spot_bumps, .2 * qty * res_table['base']['spot_price'] * res_table['base']['delta'] * (spot_bumps-1))
plt.plot()
plt.grid()
plt.show()

# END